In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import *
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE,SMOTENC
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE,RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.combine import SMOTEENN,SMOTETomek
from sklearn.decomposition import *
import mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
train=pd.read_csv(r'../input/c1-data/Training Data.csv')
test=pd.read_csv(r'../input/c1-data/Test Data.csv')
test['risk_flag']=-99
test.shape
id_col=test.id
train=train.drop('Id',axis=1)
test=test.rename(columns = {'id': 'Id'}, inplace = False)
test=test.drop('Id',axis=1)
# oversample = RandomOverSampler(sampling_strategy='minority',random_state=2021)
oversample = RandomUnderSampler(sampling_strategy='majority',random_state=2021)
# train_df=train.drop_duplicates()
# train_df=train_df.reset_index()

In [ ]:
# train_df=train[train.duplicated(['income', 'age', 'experience', 'married', 'house_ownership',
#        'car_ownership', 'profession', 'city', 'state', 'current_job_years',
#        'current_house_years'],keep='first')]
#train_df=train.drop('Id',axis=1)
data=pd.concat([train,test])
data.shape

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# a=data.drop_duplicates()
# a[a['income']==9250350]

In [ ]:
# b=data[data.duplicated()]
# b

In [ ]:
# c=data[data['income']==9250350]
# c

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data_y=data['risk_flag']
data=data.drop('risk_flag',axis=1)
data['married']=le.fit_transform(data['married'])
data['house_ownership']=le.fit_transform(data['house_ownership'])
data['car_ownership']=le.fit_transform(data['car_ownership'])
data['profession']=le.fit_transform(data['profession'])
data['city']=le.fit_transform(data['city'])
data['state']=le.fit_transform(data['state'])

In [ ]:
# col=['income', 'age', 'experience', 'married', 'house_ownership',
#        'car_ownership', 'profession', 'city', 'state', 'current_job_years',
#        'current_house_years',]
data['new_id'] = data.groupby(data.columns.tolist(), sort=False).ngroup() + 1
data['new_id1'] = data.groupby([ 'age', 'experience','profession', 'city', 'state','current_job_years'], sort=False).ngroup() + 1
data['new_id2'] = data.groupby([ 'experience','profession', 'city', 'state','current_job_years'], sort=False).ngroup() + 1
data['new_id3'] = data.groupby([ 'profession', 'city', 'state','current_job_years'], sort=False).ngroup() + 1
data['new_id4'] = data.groupby([ 'city', 'state','current_job_years','current_house_years'], sort=False).ngroup() + 1
data['new_id5'] = data.groupby([ 'city', 'state','current_job_years','income'], sort=False).ngroup() + 1
data['new_id6'] = data.groupby([ 'city', 'state','current_job_years','age'], sort=False).ngroup() + 1
data['new_id1'] = data.groupby([ 'income','age', 'experience','profession', 'city', 'state','current_job_years'], sort=False).ngroup() + 1


data=pd.concat([data,data_y],axis=1)
data.shape

In [ ]:
data.nunique()

In [ ]:
train=data[data['risk_flag']!=-99]
test=data[data['risk_flag']==-99]

In [ ]:
X=train.drop('risk_flag',axis=1)
Y=train['risk_flag']
X1,X2,Y1,Y2=train_test_split(X,Y,random_state=0,stratify=Y,shuffle=True,test_size=0.2)
X1.shape,X2.shape

In [ ]:
X1,Y1=oversample.fit_resample(X1,Y1)
Y1.value_counts()


In [ ]:
# from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
# skf=StratifiedKFold(random_state=0,n_splits=6,shuffle=True)
# score1=[]
# score2=[]
# for train_index,test_index in skf.split(X1,Y1):
#      print("TRAIN:", train_index, "TEST:", test_index)
#      x_train, x_test = X1.iloc[train_index], X1.iloc[test_index]
#      y_train, y_test = Y1.iloc[train_index], Y1.iloc[test_index]
#      model=RandomForestClassifier(random_state=0,oob_score=True)
#      model.fit(x_train,y_train)
#      pred=model.predict(x_test)
#      #print(f"OOB score on seen data is = {model.oob_score_}")
#      print(f"ROC_AUC_Score on seen data = {roc_auc_score(y_test,pred)}")
#      model.fit(X1,Y1)   
#      p=model.predict(X2)
#      #print(f"OOB score on unseen data is = {model.oob_score_}")
#      print(f"ROC_AUC_Score on unseen data = {roc_auc_score(Y2,p)}")
#      print("===============================================================")

In [ ]:
X,Y=oversample.fit_resample(X,Y)
Y.value_counts()


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
skf=StratifiedKFold(random_state=2021,n_splits=10,shuffle=True)
for train_index,test_index in skf.split(X,Y):
     print("TRAIN:", train_index, "TEST:", test_index)
     x_train, x_test = X.iloc[train_index], X.iloc[test_index]
     y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
     model=RandomForestClassifier(random_state=0,oob_score=True)
     model.fit(x_train,y_train)
     pred=model.predict(x_test)
     #print(f"OOB score on seen data is = {model.oob_score_}")
     print(f"ROC_AUC_Score = {roc_auc_score(y_test,pred)}")
     print("===============================================================")

In [ ]:
# test1=test.drop('risk_flag',axis=1)
pred=model.predict(test)
p=pd.DataFrame(pred)
p.value_counts()

In [ ]:
sub1=pd.DataFrame({'id':id_col,'risk_flag':pred})
sub1.to_csv('under_Sampled_RandomForest_1_Tree.csv',index=False)

In [ ]:
nan